# Automated Model Testing 

This notebook contains a set of automated tests for the Stroke+Rehab model.  These tests are either pass or fail and no interpretation is needed. A summary of test results is provided at the end of the notebook.

The stroke rehab model is imported within each test function because coverage measurement must begin before the module is imported and executed. When modules are imported at the top level of the notebook, all function definitions (def statements) are executed before pytest-cov starts measuring coverage, causing them to potentially be reported as uncovered lines.

**Additional testing:** Beyond the original test suite defined in our methodology, we've included extra tests to enhance validation and coverage, based on peer review feedback.

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import simpy
import sys
import statistics
from sim_tools.distributions import Lognormal
import pytest
import ipytest
from streamlit.testing.v1 import AppTest

ipytest.autoconfig()

In [3]:
# For formatting the coverage report
module_path = os.path.abspath(os.path.join('..', '..', '..'))
if module_path not in sys.path:
    sys.path.insert(0, module_path)
from helper.format_coverage import display_coverage

## Tests

### Results processing

A set of tests to check that the results of a simulated run are processed correctly for the user.

We test the processing of:

* occupancy frequencies
* probability of delay


In [4]:
@pytest.mark.parametrize('values, rel_expected, cum_expected', [
                          ([1, 1, 1, 1, 2, 2, 2, 3, 3, 4], 
                           [0.4, 0.3, 0.2, 0.1], [0.4, 0.7, 0.9, 1.0])
])
def test_result_processing_1(values, rel_expected, cum_expected):
    '''
    Test the `calculate_occupancy_frequencies` function works
    as expected.

    Expected result: relative frequencies and cumulative freqs
    are the same as expected values.

    Params:
    ------
    values: list
        list of values to test

    rel_expected: list
        list of floats - expected relative freqs

    cum_expected: list
        list of floats - expected cumulative freqs

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import calculate_occupancy_frequencies

    rel, cum, unique = calculate_occupancy_frequencies(values)
    # use all close to allow for minor floating point differences.
    assert (set(rel) == set(rel_expected)) and np.allclose(np.array(cum_expected), cum)

In [5]:
@pytest.mark.parametrize('relative, cum, p_delay_expected', [
                          ([0.4, 0.3, 0.2, 0.1], 
                           [0.4, 0.7, 0.9, 1.0], [1.0, 0.3/0.7, 0.2/0.9, 0.1/1.0])
])
def test_result_processing_2(relative, cum, p_delay_expected):
    '''
    Test the probability of delay is calculated correctly
    using the `calculate_prob_delay` function.
    
    Params:
    ------
    relative: list
        list of floats - relative freqs

    cum: list
        list of floats - cumulative freqs

    Returns:
    -------
    bool: does the function pass the test.
    '''
    from stroke_rehab_model import calculate_prob_delay

    p_delay = calculate_prob_delay(relative, cum)
    # use all close to allow for minor floating point differences.
    assert np.allclose(np.array(p_delay_expected), p_delay)

### Results collection tests

Test that the optional results collection processes for the ASU and REHAB models work correctly. 

In [6]:
def test_results_collection_1(audit_interval=1):
    '''
    Test the model collects acute stroke occupancy every day

    Expected result: len(experiment.occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit, audit_acute_occupancy

    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arr~ival rates
    default_experiment_params = Experiment()

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # optional results collection
    # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
    # MODIFIED iter 21
    env.process(audit_acute_occupancy(env, 1, audit_interval, acu_experiment, default_experiment_params))
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 21
    env.run(until=default_experiment_params.results_collection_period)

    print(f'{len(default_experiment_params.asu_occupancy)=}')
    print(f'{env.now=}')
    assert len(default_experiment_params.asu_occupancy) == (env.now - 1)

In [7]:
def test_results_collection_2(audit_interval=1):
    '''
    Test the model collects rehab occupancy every day

    Expected result: len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, audit_rehab_occupancy

    # Create the simulation environment
    env = simpy.Environment()

    # create experiment
    default_experiment = Experiment()
    
    rehab_unit = RehabilitationUnit(env, default_experiment)

    # Initialise the patient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())
    
    # Initialise the audit of rehab occupancy as a simpy process with an interval of 1 day
    # MODIFIED iter 12
    env.process(audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, default_experiment))
    
    # Run the model for the default run length in the experiment
    # MODIFIED iter 12
    env.run(until=default_experiment.results_collection_period)
    print(f'{len(default_experiment.rehab_occupancy)=}')
    print(f'{env.now=}')
    assert len(default_experiment.rehab_occupancy) == (env.now - 1)

In [8]:
def test_results_collection_system(audit_interval=1):
    '''
    SYSTEM TEST
    
    Test REHAB ward occupancy data collected is in a sensible range
    when it is connected to the ASU model.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 1 to [10-15] with sensible moments.

    Expected result: 
        len(experiment.asu_occupancy) == env.now - 1 AND
        len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit, audit_acute_occupancy, audit_rehab_occupancy

    # Create the simulation environment
    env = simpy.Environment()
    
    # a default experiment
    default_experiment_params = Experiment()

    # create models
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # Initialize the pREHAB atient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())  

    # optional results collection
    # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
    # MODIFIED iter 12
    env.process(audit_acute_occupancy(env, 1, audit_interval, acu_experiment, default_experiment_params))

    # Initialize the audit of rehab occupancy as a simpy process with an interval of 1 day
    # MODIFIED iter 12
    env.process(audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, default_experiment_params))
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 12
    env.run(until=default_experiment_params.results_collection_period)

    # print info for debug
    print(f'{len(default_experiment_params.asu_occupancy)=}')
    print(f'{len(default_experiment_params.rehab_occupancy)=}')
    print(f'{env.now=}')

    # test
    assert len(default_experiment_params.asu_occupancy) == (env.now - 1) and \
        len(default_experiment_params.rehab_occupancy) == (env.now - 1)

## Model run tests

Here we test that various modes of running the model work correctly.  These include

* results collection period
* warm-up
* single run mode
* repeatable results using random number sets.

In [9]:
def test_results_collection_system(audit_interval=1):
    '''
    SYSTEM TEST
    
    Test REHAB ward occupancy data collected is in a sensible range
    when it is connected to the ASU model.
    
    Expected result: The type collected is int. The values are in 
    the range in the range 1 to [10-15] with sensible moments.

    Expected result: 
        len(experiment.asu_occupancy) == env.now - 1 AND
        len(experiment.rehab_occupancy) == env.now - 1

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit, audit_acute_occupancy, audit_rehab_occupancy

    # Create the simulation environment
    env = simpy.Environment()
    
    # a default experiment
    default_experiment_params = Experiment()

    # create models
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # Initialize the pREHAB atient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())  

    # optional results collection
    # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
    # MODIFIED iter 12
    env.process(audit_acute_occupancy(env, 1, audit_interval, acu_experiment, default_experiment_params))

    # Initialize the audit of rehab occupancy as a simpy process with an interval of 1 day
    # MODIFIED iter 12
    env.process(audit_rehab_occupancy(env, 1, audit_interval, rehab_unit, default_experiment_params))
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 12
    env.run(until=default_experiment_params.results_collection_period)

    # print info for debug
    print(f'{len(default_experiment_params.asu_occupancy)=}')
    print(f'{len(default_experiment_params.rehab_occupancy)=}')
    print(f'{env.now=}')

    # test
    assert len(default_experiment_params.asu_occupancy) == (env.now - 1) and \
        len(default_experiment_params.rehab_occupancy) == (env.now - 1)

In [10]:
@pytest.mark.parametrize('warm_up, audit_interval', [
                          (365, 1),
                          (1000, 1)
])
def test_warm_up(warm_up, audit_interval):
    '''
    Test warm-up works correctly for ASU+REHAB ward occupancy

    Expected result: 
        len(experiment.asu_occupancy) == experiment.results_collection_period 
        AND len(experiment.rehab_occupancy) == experiment.results_collection_period 

    Params:
    ------
    audit_interval: 1
        duration of audit.

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit, audit_acute_occupancy, audit_rehab_occupancy

    # Create the simulation environment
    env = simpy.Environment()
    
    # a default experiment
    default_experiment_params = Experiment(warm_up=warm_up)

    # create models
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # Initialize the pREHAB atient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())  

    # optional results collection
    # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
    env.process(audit_acute_occupancy(env, warm_up, audit_interval, acu_experiment, default_experiment_params))

    # Initialize the audit of rehab occupancy as a simpy process with an interval of 1 day
    env.process(audit_rehab_occupancy(env, warm_up, audit_interval, rehab_unit, default_experiment_params))
    
    # Run the simulation until the specified run length in the Experiment parameters
    env.run(until=default_experiment_params.warm_up + default_experiment_params.results_collection_period)

    # print info for debug
    print(f'{len(default_experiment_params.asu_occupancy)=}')
    print(f'{len(default_experiment_params.rehab_occupancy)=}')
    print(f'{env.now=}')
    print(f'{default_experiment_params.results_collection_period=}')
    print(f'{default_experiment_params.warm_up + default_experiment_params.results_collection_period=}')

    # test
    assert len(default_experiment_params.asu_occupancy) == (default_experiment_params.results_collection_period) and \
        len(default_experiment_params.rehab_occupancy) == (default_experiment_params.results_collection_period) 

In [11]:
def test_single_run():
    '''
    Test the the single_run function returns a dictionary of 
    results.

    The results dictionary contains the following keys:

    'relative_freq_asu'
    'prob_delay_asu'
    'unique_vals_asu'
    'relative_freq_rehab'
    'prob_delay_rehab'
    'unique_vals_rehab'

    Expected result: 
        len(run_results) == 6 and type(run_results) == dict

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, single_run

    # a default experiment
    default_experiment_params = Experiment()

    # run the model
    run_results = single_run(default_experiment_params)

    print(f"{run_results['relative_freq_asu']=}")
    
    # test
    assert len(run_results) == 6 and type(run_results) == dict

### Random number set test (ASU only)

Test that ASU results are repeated each time the same random number set is used.

In [12]:
@pytest.mark.parametrize('random_number_set, print_output', [
                          (0, False),
                          (1, False),
                          (2, False),
                          (101, False),
                          (42, False),
])
def test_random_number_set_1(random_number_set, print_output):
    '''
    Test the the ASU model produces repeatable results. 

    Compares
    min, max, mean of occupancy.
    
    Expected result: 
        set(run1) == set(run2)

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit, audit_acute_occupancy

    results = []

    for i in range(2):
        
        # Create the simulation environment
        env = simpy.Environment()
        
        # Initialize the Acute Stroke Unit model#
        # set ASU arr~ival rates
        experiment = Experiment(random_number_set=random_number_set)
    
        # modified iteration 19. 
        # add RU, but do not run the model
        rehab_unit = RehabilitationUnit(env, experiment)
        
        asu = AcuteStrokeUnit(env, experiment, rehab_unit)
        
        # Start the patient generators for each type of patient
        env.process(asu.stroke_patient_generator())
        env.process(asu.tia_patient_generator())
        env.process(asu.neuro_patient_generator())
        env.process(asu.other_patient_generator())
    
        # optional results collection
        # Start the audit_acute_occupancy generator function to record ASU occupancy at intervals
        env.process(audit_acute_occupancy(env, 1, 1, asu, experiment))
        
        # Run the simulation until the specified run length in the Experiment parameters
        env.run(until=experiment.results_collection_period)

        if print_output: 
            print(f'Run {i} results:')
            print(f'{min(experiment.asu_occupancy)=}')
            print(f'{max(experiment.asu_occupancy)=}')
            print(f'{statistics.fmean(experiment.asu_occupancy)=}')
            print([round(q, 1) for q in statistics.quantiles(experiment.asu_occupancy, n=10)])

        results.append(set((min(experiment.asu_occupancy), 
                            max(experiment.asu_occupancy),
                            statistics.fmean(experiment.asu_occupancy))))
    
    # test
    assert results[0] == results[1]

In [13]:
@pytest.mark.parametrize('random_number_set, print_output', [
                          (0, False),
                          (1, False),
                          (2, False),
                          (101, False),
                          (42, False),
])
def test_random_number_set_2(random_number_set, print_output):
    '''
    Test the the single_run function returns a dictionary of 
    results.

    The results dictionary contains the following keys:

    'relative_freq_asu'
    'prob_delay_asu'
    'unique_vals_asu'
    'relative_freq_rehab'
    'prob_delay_rehab'
    'unique_vals_rehab'

    Expected result: 
        len(run_results) == 6 and type(run_results) == dict

    Returns:
    -------
    bool: does the model pass the test.
    '''
    from stroke_rehab_model import Experiment, single_run

    results = []
    for i in range(2):
    
        # set a random number set for streams
        experiment = Experiment(random_number_set=random_number_set)
    
        # run the model
        run_results = single_run(experiment)

        if print_output: 
            print(f'Run {i} results:')
            print(f'{min(experiment.asu_occupancy)=}')
            print(f'{max(experiment.asu_occupancy)=}')
            print(f'{statistics.fmean(experiment.asu_occupancy)=}')
            print([round(q, 1) for q in statistics.quantiles(experiment.asu_occupancy, n=10)])

        results.append(set((min(experiment.asu_occupancy), 
                            max(experiment.asu_occupancy),
                            statistics.fmean(experiment.asu_occupancy))))
    
    # test
    assert results[0] == results[1]

### Lognormal test

Test that lognomal function correctly calculates the moments of the underlying normal dist.

In [14]:
@pytest.mark.parametrize('mean, std', [
                          (128.79, 267.51),
                          (50.0, 2.0),
                          (10.5, 1.0),
])
def test_lognormal_moments(mean, std):
    '''
    Test that lognomal function correctly calculates 
    the moments of the underlying normal dist.

    Params:
    ------
    mean: float
        mean of the lognormal distribution

    std: float
        st dev of the lognormal distribution

    Returns:
    -------
    bool
    '''
    from stroke_rehab_model import normal_moments_from_lognormal
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)
    
    ## llm code
    llm_mu, llm_sigma = normal_moments_from_lognormal(mean, std)
    print(llm_mu, llm_sigma)
    
    assert (llm_mu, llm_sigma) == pytest.approx((expected_moments.mu, expected_moments.sigma))

### Extreme value tests

Extreme value tests are used pragmatically to block of routes/arrivals/activites in the simulation model and check the results.

The most simple way to modify the model for these tests is to set parameters to $M$ a very large number.

We test

* Block all arrivals
* Acute LoS is infinite
* Block all but stroke->rehab patient arrivals in the ASU.
* Block all rehab arrivals apart from stroke.
* Block all external rehab arrivals
* Rehab LoS is infinite (Rehab model only)
* Block all arrivals to the ASU and Rehab models

In [15]:
M = 10_000_000

In [16]:
def test_ev_2(large_number=M):
    '''
    All patient types have have their inter-arrival time is 
    set to $M$ a very large number
    
    Expected result: No patients arrive to the model.
    
    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    int: the number of patients that arrived to the model.
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        stroke_mean=large_number,
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number,
        trace=False,
    )

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 12
    env.run(until=default_experiment_params.results_collection_period)

    assert acu_experiment.patient_count == 0

In [17]:
def test_ev_3(large_number=M):
    '''
    All patient types have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: No patients depart the model 
    The number of arrivals = the occupancy of the model.

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_mean=large_number,  # stroke->rehab
        esd_mean=large_number,    # stroke-> ESD
        other_dest_mean=large_number, # stroke->other
        tia_dest_mean=large_number,   # TIA patients
        neuro_dest_mean=large_number, # complex neuro patients
        other_dest_mean_2=large_number, # other (med outlier) patients
        trace=False,
    )

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 21
    env.run(until=default_experiment_params.results_collection_period)

    print(f'{acu_experiment.patient_count=}')
    print(f'{acu_experiment.occupancy=}')
    
    assert acu_experiment.patient_count == acu_experiment.occupancy 

In [18]:
def test_ev_4(large_number=M):
    '''
    All patient types apart from stroke-rehab patients
    have have their mean length 
    of stay time set to $M$ a very large number
    
    Expected result: Only stroke patients depart the
    model.
    (assessed crudely with patient_count > occupancy),
    and also from log - see manual testing notebook.
    
    Params:
    -------
    large_number: int
        M a very large number 
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_mean=7.4,  # stroke->rehab
        esd_mean=large_number,    # stroke-> ESD
        other_dest_mean=large_number, # stroke->other
        tia_dest_mean=large_number,   # TIA patients
        neuro_dest_mean=large_number, # complex neuro patients
        other_dest_mean_2=large_number, # other (med outlier) patients
        trace=False,
    )

    # modified iteration 19. 
    # add RU, but do not run the model
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 21
    env.run(until=default_experiment_params.results_collection_period)

    print(f'{acu_experiment.patient_count=}')
    print(f'{acu_experiment.occupancy=}')
    
    assert acu_experiment.patient_count > acu_experiment.occupancy

In [19]:
def test_ev_5(large_number=M):
    '''
    Complex Neuro, Other, have their rehab inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the rehab model 
    is "Stroke". This is verified by the patient counts variables in the model.

    Notes:
    ------
    This test will need to be modified when the hardcoded parameters
    are migrated to the Experiment class. 

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    bool: rehab_unit.stroke_count == rehab_unit.patient_count
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # set Rehab arrival rates
    default_experiment_params = Experiment(
        rehab_neuro_iat = large_number,
        rehab_other_iat = large_number,
    )
    
    # this will be need to be modified in subsequent iterations
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
        
    # Start the patient generators for each type of patient in the RehabilitationUnit instance
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())
        
    # Run the simulation until the specified run length
    env.run(until=1825)

    # print out stats collected
    print(f'{rehab_unit.patient_count=}')
    print(f'{rehab_unit.stroke_count=}')
    print(f'{rehab_unit.neuro_count=}')
    print(f'{rehab_unit.other_count=}')
    
    assert rehab_unit.stroke_count == rehab_unit.patient_count

In [20]:
def test_ev_6(large_number=M):
    '''
    All patient types have their REHAB inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: No patients arrive to the model
    This is verified by the patient count variables in the model.

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    bool:rehab_unit.patient_count == 0
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # set Rehab arrival rates
    default_experiment_params = Experiment(
        rehab_stroke_iat = large_number,
        rehab_neuro_iat = large_number,
        rehab_other_iat = large_number,
    )
    
    # this will be need to be modified in subsequent iterations
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient in the RehabilitationUnit instance
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())
        
    # Run the simulation until the specified run length
    env.run(until=1825)

    # print out stats collected
    print(f'{rehab_unit.patient_count=}')
    print(f'{rehab_unit.stroke_count=}')
    print(f'{rehab_unit.neuro_count=}')
    print(f'{rehab_unit.other_count=}')
    print(f'{rehab_unit.occupancy=}')
    
    assert rehab_unit.patient_count == 0

In [21]:
def test_ev_7(large_number=M):
    '''
    All patient types have have their mean length 
    of stay time in REHAB set to $M$ a very large number
    
    Expected result: No patients depart the rehab model 
    The occupancy of the model is equal to the no. patient arrivals
    
    Notes:
    -----
    This test will also need to be modified when TIA treatment is added
    and tested when working in connection with the ASU.

    Params:
    -------
    large_number: int
        M a very large number 
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model#
    # set ASU arrival rates
    default_experiment_params = Experiment(
        rehab_stroke_esd_mean=large_number, # stroke->ESD
        rehab_stroke_other_mean=large_number, # stroke->other
        rehab_neuro_mean=large_number, # complex neuro
        rehab_other_mean=large_number, # other patients.
        trace=False,   
    )

    # this will be need to be modified in subsequent iterations
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    
    # Start the patient generators for each type of patient in the instance
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())
        
    # Run the simulation until the specified run length
    env.run(until=100)

    # print out stats collected
    print(f'{rehab_unit.patient_count=}')
    print(f'{rehab_unit.stroke_count=}')
    print(f'{rehab_unit.neuro_count=}')
    print(f'{rehab_unit.other_count=}')
    print(f'{rehab_unit.occupancy=}')
    
    assert rehab_unit.patient_count == rehab_unit.occupancy

In [22]:
def test_ev_8(large_number=M):
    '''
    All patient types have their AUS and REHAB inter-arrival 
    time is set to $M$ a very large number
    
    Expected result: No patients arrive to the model
    This is verified by the patient count variables in the model.

    Params:
    -------
    large_number: int
        M a very large number 

    Returns:
    --------
    bool:rehab_unit.patient_count == 0 and asu.patient_count == 0
    '''
    from stroke_rehab_model import Experiment, RehabilitationUnit, AcuteStrokeUnit

    # Create the simulation environment
    env = simpy.Environment()
    
    # set Rehab arrival rates
    default_experiment_params = Experiment(
        stroke_mean=large_number,
        tia_mean=large_number,
        neuro_mean=large_number,
        other_mean=large_number,
        rehab_stroke_iat = large_number,
        rehab_neuro_iat = large_number,
        rehab_other_iat = large_number,
    )
    
    # create models
    rehab_unit = RehabilitationUnit(env, default_experiment_params)
    acu_experiment = AcuteStrokeUnit(env, default_experiment_params, rehab_unit)
    
    # Start the ASU patient generators for each type of patient
    env.process(acu_experiment.stroke_patient_generator())
    env.process(acu_experiment.tia_patient_generator())
    env.process(acu_experiment.neuro_patient_generator())
    env.process(acu_experiment.other_patient_generator())

    # Initialize the pREHAB atient generators as simpy processes
    env.process(rehab_unit.stroke_patient_generator())
    env.process(rehab_unit.neuro_patient_generator())
    env.process(rehab_unit.other_patient_generator())  

    # no optional data collection needed in this test
    
    # Run the simulation until the specified run length in the Experiment parameters
    # MODIFIED iter 21
    env.run(until=default_experiment_params.results_collection_period)

    # print out stats collected
    print(f'{acu_experiment.patient_count=}')
    print(f'{rehab_unit.patient_count=}')
    print(f'{rehab_unit.stroke_count=}')
    print(f'{rehab_unit.neuro_count=}')
    print(f'{rehab_unit.other_count=}')
    print(f'{rehab_unit.occupancy=}')
    
    assert rehab_unit.patient_count == 0 and acu_experiment.patient_count== 0

### Test tracing functionality

In [23]:
def test_trace(capsys):
    """
    Verify trace messages are printed when trace=True.
    """
    from stroke_rehab_model import Experiment, multiple_replications

    # Run the model once
    default_experiment = Experiment(trace=True)
    _ = multiple_replications(default_experiment, num_replications=1)

    # Capture printed output
    captured = capsys.readouterr()
    
    # Check for some of the expected messages based on actual code
    assert "Stroke patient" in captured.out
    assert "finished treatment at" in captured.out

### Test plotting functions

In [24]:
def test_occupancy_plot():
    """
    Check the output of occupancy_plot() is as expected.
    """
    from stroke_rehab_model import occupancy_plot

    # Create a plot with simple data
    relative_frequency = np.array([0.2, 0.5, 0.3])
    unique_values = np.array([0, 1, 2])
    fig, ax = occupancy_plot(relative_frequency, unique_values)

    # Check characteristics
    assert len(ax.patches) == 3  # Should have 3 bars
    assert ax.patches[1].get_height() == 0.5  # Check bar height
    assert ax.get_xlabel() == "No. people in ASU"  # Default label
    assert ax.get_ylabel() == "Relative Frequency"
    assert isinstance(fig, plt.Figure)
    assert isinstance(ax, plt.Axes)

    # Close the figure to prevent display
    plt.close(fig)

In [25]:
def test_prob_delay_plot():
    """
    Check the output of prob_delay_plot() is as expected.
    """
    from stroke_rehab_model import prob_delay_plot

    # Create a plot with simple data
    prob_delay = np.array([0.1, 0.6, 0.9])
    unique_values = np.array([0, 5, 10])
    fig, ax = prob_delay_plot(prob_delay, unique_values)
    
    # Check characteristics
    assert len(ax.lines) == 1  # Should have one step line
    assert ax.get_xlabel() == "No. acute beds available"  # Default label
    assert ax.get_ylabel() == "Probability of Delay"
    assert isinstance(fig, plt.Figure)
    assert isinstance(ax, plt.Axes)

    # Close the figure to prevent display
    plt.close(fig)

### Test streamlit app

In [26]:
def test_streamlit_app_loads():
    """
    Verify that the streamlit app loads without errors.
    """
    # Run the app
    at = AppTest.from_file("stroke_rehab_interface.py")
    at.run()
    
    # Expected title
    exp_title = ("A modelling tool for capacity planning in acute and " +
                 "community stroke services")

    # Check that main elements are present
    assert not at.exception
    assert exp_title in str(at.title.values)
    assert len(at.number_input) == 6  # Patients, warm-up and replications
    assert len(at.checkbox) == 1  # Trace checkbox
    assert len(at.button) == 1  # Simulate button

In [27]:
def test_streamlit_simulate_button():
    """
    Check that clicking the simulate button runs the simulation.
    """
    # Run the app
    at = AppTest.from_file("stroke_rehab_interface.py")
    at.run()

    # Run for 2 replications (else test times out)
    at.number_input[0].set_value(2).run()

    # Click simulate button
    at.button[0].click().run()

    # Check that no exception occurred
    assert not at.exception

    # Get dataframes
    acu_df = at.table[0].value
    rehab_df = at.table[1].value
    
    # Expected values
    expected_columns = ['p(delay)', '1 in every n patients delayed']
    expected_acu_index = [9, 10, 11, 12, 13, 14]
    expected_rehab_index = [10, 11, 12, 13, 14, 15, 16]
    
    # Check columns (same for both dataframes)
    assert list(acu_df.columns) == expected_columns
    assert list(rehab_df.columns) == expected_columns
    
    # Check index values
    assert list(acu_df.index) == expected_acu_index
    assert list(rehab_df.index) == expected_rehab_index

## Run all automated tests

In [28]:
ipytest.run(
    "-vv", "--no-header",
    "--cov=stroke_rehab_model", "--cov=stroke_rehab_interface",
    "--cov-report=term", "--cov-report=annotate"
)

======================================= test session starts ========================================
collecting ... collected 33 items

t_01f77a056be94fb6ad501e29dbbc9c15.py::test_result_processing_1[values0-rel_expected0-cum_expected0] PASSED [  3%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_result_processing_2[relative0-cum0-p_delay_expected0] PASSED [  6%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_results_collection_1 PASSED                      [  9%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_results_collection_2 PASSED                      [ 12%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_results_collection_system PASSED                 [ 15%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_warm_up[365-1] PASSED                            [ 18%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_warm_up[1000-1] PASSED                           [ 21%]
t_01f77a056be94fb6ad501e29dbbc9c15.py::test_single_run PASSED                                [ 24%]
t_01f77a056be94fb6ad501e29dbbc9c15.py

<ExitCode.OK: 0>

## Test coverage report

The percentage coverage is reported in the cell above. Below is the annotated report for the model. Grey lines (" ") are excluded, green lines (">") are covered, and red lines ("!") are not covered.

In [29]:
display_coverage('stroke_rehab_model.py,cover')

In [30]:
display_coverage('stroke_rehab_interface.py,cover')